In [7]:
from scipy.spatial import distance
from imutils import face_utils
import dlib
import cv2
import tkinter as tk
from tkinter import filedialog
import numpy as np

# 弹窗提示消息
def alert_popup(title, message):
    root = tk.Tk()
    root.title(title)
    w = tk.Label(root, text=message, width=120, height=10)
    w.pack()
    b = tk.Button(root, text="OK", command=root.destroy, width=10)
    b.pack()
    root.mainloop()

# 计算嘴部纵横比
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[2], mouth[10])
    B = distance.euclidean(mouth[4], mouth[8])
    C = distance.euclidean(mouth[0], mouth[6])
    mar = (A + B) / (2.0 * C)
    return mar

# 设置一些初始变量
thresh = 0.54   # MAR值的阈值，超过这个值认为嘴巴张开
yawn_frames = 20 # 连续多少帧判断为哈欠
yawn_counter = 0  # 嘴巴张开的连续帧数计数器
yawn_alerted = False # 是否已经有打哈欠的提示
detect = dlib.get_frontal_face_detector()
alarm_on=False
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
(mouth_start, mouth_end) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

file_path = filedialog.askopenfilename(title='请选择一个视频文件', filetypes=[("Video files", "*.mp4 *.avi")])
if not file_path:
    exit()  # 如果没有选中文件，直接退出程序

cap = cv2.VideoCapture(file_path)
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('yawn_res.avi', fourcc, fps, size)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 转换当前帧为灰度图像
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 自适应直方图均衡化
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_gray = clahe.apply(gray)

    # 对CLAHE结果应用高斯模糊，以减少噪声
    blur_sigma = 2  # 高斯核的标准偏差，在不同图像上可能需要调整
    gaussian_blur = cv2.GaussianBlur(clahe_gray, (5, 5), blur_sigma)

    # 使用Canny算法进行边缘检测
    low_threshold = 50  # Canny边缘检测的低阈值
    high_threshold = 150  # Canny边缘检测的高阈值
    canny_edges = cv2.Canny(gaussian_blur, low_threshold, high_threshold)

    # 进行形态学变换 - 闭操作
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(canny_edges, cv2.MORPH_CLOSE, kernel)

    # 将闭操作后的边缘与原CLAHE图像进行叠加
    # 转换闭操作后的边缘图像的数据类型，使其可以与原图像进行加权叠加
    closing_for_combination = cv2.convertScaleAbs(closing)
    enhanced_edges = cv2.addWeighted(clahe_gray, 0.7, closing_for_combination, 0.3, 0)
    faces = detect(enhanced_edges, 0)

    for face in faces:
        landmarks = predict(gray, face)
        landmarks_np = face_utils.shape_to_np(landmarks)
        mouth = landmarks_np[mouth_start:mouth_end]
        mar = mouth_aspect_ratio(mouth)

        # 绘制嘴巴的矩形框
        cv2.rectangle(frame, tuple(mouth[0]), tuple(mouth[6]), (0, 255, 0), 2)

        if mar > thresh:
            yawn_counter += 1
            # if yawn_counter >= yawn_frames and not yawn_alerted:
            #     alert_popup("Yawn Alert", "You're yawning, take a break!")
            #     yawn_alerted = True
            if yawn_counter >= yawn_frames:
                if not alarm_on:
                    alarm_on = True
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            # if yawn_alerted:
            #     yawn_alerted = False
            yawn_counter = 0
            #blink_counter = 0
            alarm_on = False

        cv2.putText(frame, "MAR: {:.2f}".format(mar), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    out.write(frame)
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()